<a href="https://colab.research.google.com/github/AsafShiloah/LLM_Biase_Manipulation/blob/main/lm_harness_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git@big-refactor
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install llama-cpp-python
!pip install langchain
!pip install langchain-community
!pip install langchain llama-cpp-python
!pip install xlsxwriter
!pip install openpyxl
!pip install datasets --upgrade
!pip install lm-eval
!pip install bitsandbytes

  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git (to revision big-refactor) to /tmp/pip-req-build-p8evxspw
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-p8evxspw
  Running command git checkout -b big-refactor --track origin/big-refactor
  Switched to a new branch 'big-refactor'
  Branch 'big-refactor' set up to track remote branch 'big-refactor' from 'origin'.
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit 967eb4fa90b80ba4e8cc7a2fd171f44f0e384808
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from datasets import load_dataset
from huggingface_hub import notebook_login
from openpyxl import load_workbook
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles import PatternFill
import pandas as pd
import os
import evaluate
from langchain import LLMChain, PromptTemplate
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
!rm -rf /content/LLM-Research-Technion
!git clone https://github.com/yotamgardosh/LLM-Research-Technion.git
os.chdir('LLM-Research-Technion')
from models_class import Model
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Cloning into 'LLM-Research-Technion'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 89 (delta 35), reused 35 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 8.95 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [3]:
mistral_7b = Model("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF","Mistral_7b","openhermes-2.5-mistral-7b.Q4_K_M.gguf")
orca_2_7b = Model("TheBloke/Orca-2-7B-GGUF","Orca2_7b","orca-2-7b.Q4_K_M.gguf")
llama_2_7b = Model("TheBloke/Llama-2-7B-GGUF","Llama2_7b","llama-2-7b.Q4_K_M.gguf")

------------------------------------------------------------------------
Initialized model 'Mistral_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
------------------------------------------------------------------------



------------------------------------------------------------------------
Initialized model 'Orca2_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
--------------

In [4]:

ds = load_dataset("rabell/SysEngBench")
mistral_path = mistral_7b.create_model_path()
orca_path = orca_2_7b.create_model_path()
llama_path = llama_2_7b.create_model_path()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
print(mistral_path)
print(orca_path)
print(llama_path)

/root/.cache/huggingface/hub/models--TheBloke--OpenHermes-2.5-Mistral-7B-GGUF/snapshots/5682e25bb033d9d21f6d159859e21df4552c1f26/openhermes-2.5-mistral-7b.Q4_K_M.gguf
/root/.cache/huggingface/hub/models--TheBloke--Orca-2-7B-GGUF/snapshots/13f4f3f200ee07ba0ac1edf0369a24d2050da7b4/orca-2-7b.Q4_K_M.gguf
/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-GGUF/snapshots/b4e04e128f421c93a5f1e34ac4d7ca9b0af47b80/llama-2-7b.Q4_K_M.gguf


In [6]:
YAML_SysEngBench_string = '''
task: demo_sysengbench_zero_shot
dataset_path: rabell/SysEngBench
dataset_name: default  # Replace with specific subset name if applicable
description: "The following are multiple-choice questions (with answers) related to System Engineering."
test_split: test
output_type: multiple_choice
doc_to_text: |
  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
'''

with open('/content/mmlu_SysEngBench.yaml', 'w') as f:
    f.write(YAML_SysEngBench_string)


In [9]:
import datasets
import evaluate
def load_metric(metric_name, **kwargs):
    return evaluate.load(metric_name, **kwargs)

# Monkey-patch datasets.load_metric
datasets.load_metric = load_metric

accuracy = evaluate.load("accuracy")
!lm_eval \
    --model hf \
    --model_args model_path=mistral_path\
    --include_path ./ \
    --tasks demo_sysengbench_zero_shot \
    --limit 100 \
    --output output/demo_sysengbench_zero_shot/ \
    --log_samples


2024-09-26:10:06:22,323 INFO     [utils.py:145] Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-09-26:10:06:22,323 INFO     [utils.py:148] Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-09-26:10:06:22,323 INFO     [utils.py:161] NumExpr defaulting to 16 threads.
2024-09-26:10:06:22,475 INFO     [config.py:54] PyTorch version 2.4.0+cpu available.
2024-09-26:10:06:22,475 INFO     [config.py:101] TensorFlow version 2.17.0 available.
2024-09-26:10:06:22,477 INFO     [config.py:114] JAX version 0.4.33 available.
2024-09-26 10:06:23.823731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 10:06:23.844098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register